# Ajuste hyperparametros LSTM

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras import layers, models
import tensorflow as tf
import math
import os
import json

import random as python_random

from keras_tuner import HyperModel, HyperParameters
from keras_tuner.tuners import RandomSearch
from keras_tuner.tuners import Hyperband
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.metrics import MeanAbsolutePercentageError

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error

np.random.seed(123)
python_random.seed(123)
tf.random.set_seed(1234)

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

from funcionesComunes import *

# Funciones

In [2]:
class LSTMHyperModel(HyperModel):
    
    def __init__(self, input_shape, num_lstm_layers_input=3, num_lstm_layers_after_input=3):
        self.input_shape = input_shape
        self.num_lstm_layers_input = num_lstm_layers_input
        self.num_lstm_layers_after_input = num_lstm_layers_after_input
    
    def build(self, hp):
        
        # Podemos ver como "hp.Int" permite ir modificando los parámetros del modelo con INT
        # Con "hp.Float" lo hace pero para valores decimales 
        # IMPORTANTE!!! Nunca se devuelve el return_sequences si la LSTM es la última capa
        # IMPORTANTE!!! Si estamos aplicando capas LSTM despues Dropout y luego LSTM de nuevo, debemos de poner en todas las primeras capas LSTM return_sequences=True !!!IMPORTANTE
        
        model = Sequential()
        
        # Número de capas LSTM
        num_lstm_layers = hp.Int('num_lstm_layers', min_value=1, max_value=self.num_lstm_layers_input, default = self.num_lstm_layers_input)

        for i in range(num_lstm_layers):
            # Añadiendo capas LSTM
            # Solo la primera capa necesita input_shape
            if i == 0:
                
                # Se tiene una capa LSTM que empieza con 32 unidades y va aumentando de 32 en 32 hasta 128
                model.add(LSTM(units=hp.Int(f'units_lstm_{i}', min_value=32, max_value=128, step=32),
                               input_shape=self.input_shape,
                               return_sequences=True,  # True si hay más de una capa LSTM
                               use_bias=True))
            else:
                model.add(LSTM(units=hp.Int(f'units_lstm_{i}', min_value=32, max_value=128, step=32),
                               return_sequences=True,  # True para todas excepto la última capa LSTM
                               use_bias=True))
        
        # Se modifica la capa Dropout desde 0.0 hasta 0.5 con un step de 0.05
        model.add(Dropout(hp.Float('dropout', min_value=0.0, max_value=0.5, default=0.25, step=0.05)))
        
        # Creamos un bucle que permite incrementar las capas LSTM destras de la capa Dropout
        num_lstm_layers_after = hp.Int('num_lstm_layers_after', min_value=1, max_value=self.num_lstm_layers_after_input, default = self.num_lstm_layers_after_input)
        
        for i in range(num_lstm_layers_after):
            # Añadiendo capas LSTM después de Dropout
            model.add(LSTM(units=hp.Int(f'units_lstm_after_{i}', min_value=32, max_value=128, step=32),
                           return_sequences=(i < num_lstm_layers_after - 1),  # True para todas excepto la última capa LSTM
                           use_bias=True))
        
        # Capa dense que empieza con 4 unidades y va aumentando de 4 en 4 hasta 64
        model.add(Dense(hp.Int('dense_units', min_value=4, max_value=64, step=4), activation=hp.Choice('dense_activation',values=['relu', 'sigmoid', 'tanh', 'elu', 'relu'],default='relu'), use_bias=True))
        
        # Se modifica la capa Dropout desde 0.0 hasta 0.5 con un step de 0.05
        model.add(Dropout(hp.Float('dropout_2', min_value=0.0, max_value=0.5, default=0.25, step=0.05)))

        model.add(Dense(1, activation=hp.Choice('dense_activation',values=['relu', 'sigmoid', 'tanh', 'elu', 'relu'],default='relu'), use_bias=True))
        
        # Variaciones de los optimizadores
        optimizer = hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop'])
        if optimizer == 'adam':
            opt = Adam(
                learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
            )
        elif optimizer == 'sgd':
            opt = SGD(
                learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
            )
        else: # rmsprop
            opt = RMSprop(
                learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
            )
        
        
        model.compile(optimizer=opt,
                      loss=MeanSquaredError(),
                      metrics=[
                        MeanSquaredError(name='mse'),
                        RootMeanSquaredError(name='rmse'),
                        MeanAbsolutePercentageError(name='mape')
                        ]
                    )
        
        return model
    
    def fit(self, hp, model, *args, **kwargs):
        return model.fit(*args,
            batch_size=hp.Choice("batch_size", [16, 24, 32]),
            **kwargs)

# Bucle de ajuste de hiperparametros

In [ ]:
for nombreArchivo in os.listdir("../Datasets/RCPMerged"):

    print("Procesando el archivo: ", nombreArchivo)

    # Supongamos que tus datos están en un archivo CSV
    df = pd.read_csv(f'../Datasets/RCPMerged/{nombreArchivo}')

    # Ver las primeras filas
    df = codification(df)

    # Normalizamos los datos de crecimiento de los individuos
    df, valorNormalizacion = individualNormalization(df)

    # División adicional para validación
    train_data, val_data, test_data = split_population_individuals(df, train_pct=0.80, val_pct_in_train=0.20, details=False)
    train_data.shape, val_data.shape, test_data.shape

    WINDOWS_SIZE = 3

    # Obtenemos X e y para los datasets de train, val y test 
    X_train, y_train = df_to_X_y_ind_3(train_data, WINDOWS_SIZE)
    X_val, y_val = df_to_X_y_ind_3(val_data, WINDOWS_SIZE)
    X_test, y_test = df_to_X_y_ind_3(test_data, WINDOWS_SIZE)
    print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

    # Creación de modelo de ajuste de hiperparámetros
    hypermodel = LSTMHyperModel(input_shape=(WINDOWS_SIZE+1, X_train.shape[2]), num_lstm_layers_input=3, num_lstm_layers_after_input=3)

    tuner = Hyperband(
        hypermodel,
        objective='val_loss',
        max_epochs=50,
        factor=3,
        directory= 'resultadosModelosLSTM',
        project_name="resultadosModelosLSTM/" + nombreArchivo[:-4],
    )
    
    print(X_train.shape, X_test.shape, X_val.shape)

    # Búsquedas de hiperparámetros
    tuner.search(X_train, y_train, epochs=200, validation_data=(X_val, y_val), 
                    callbacks=[EarlyStopping(monitor='val_loss', patience=10)])

    # Obtener los 10 mejores hiperparámetros
    best_hps = tuner.get_best_hyperparameters(num_trials=10)

    # Obtener los 10 mejores modelos
    best_models = tuner.get_best_models(num_models=10)

    for i, trial in enumerate(best_models):
        print(f"Modelo numero {i+1}\n")

        # Obtener el modelo y los hiperparámetros correspondientes
        model = best_models[i]
        hps = best_hps[i]
        batch_size_LSTM = hps.get('batch_size')

        # Mostrar el resumen del modelo
        print("Resumen del modelo:")
        model.summary()

        # Entrenar el modelo (hacer fit) con los datos de entrenamiento
        # Ajustar el número de epochs y callbacks según sea necesario
        model.fit(X_train, y_train, epochs=2, validation_data=(X_val, y_val), 
                    callbacks=[EarlyStopping(monitor='val_loss', patience=10)])

        # Entrenamiento
        predictions_train = predictionForIndividuals(X_train, y_train, model, batch_size_LSTM)
        predictions_train["PredictionsDenormalize"] = predictions_train.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Predictions"), axis=1)
        predictions_train["ActualDenormalize"] = predictions_train.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Actuals"), axis=1)

        train_mse = mean_squared_error(predictions_train["ActualDenormalize"], predictions_train["PredictionsDenormalize"])
        train_rmse = np.sqrt(train_mse)
        train_mape = mean_absolute_percentage_error(predictions_train["ActualDenormalize"], predictions_train["PredictionsDenormalize"]) *100
        train_r2 = r2_score(predictions_train["ActualDenormalize"], predictions_train["PredictionsDenormalize"])

        # Validación
        predictions_val = predictionForIndividuals(X_val, y_val, model, batch_size_LSTM)
        predictions_val["PredictionsDenormalize"] = predictions_val.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Predictions"), axis=1)
        predictions_val["ActualDenormalize"] = predictions_val.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Actuals"), axis=1)

        val_mse = mean_squared_error(predictions_val["ActualDenormalize"], predictions_val["PredictionsDenormalize"])
        val_rmse = np.sqrt(val_mse)
        val_mape = mean_absolute_percentage_error(predictions_val["ActualDenormalize"], predictions_val["PredictionsDenormalize"]) *100
        val_r2 = r2_score(predictions_val["ActualDenormalize"], predictions_val["PredictionsDenormalize"])

        # Prueba
        predictions_test = predictionForIndividuals(X_test, y_test, model, batch_size_LSTM)
        predictions_test["PredictionsDenormalize"] = predictions_test.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Predictions"), axis=1)
        predictions_test["ActualDenormalize"] = predictions_test.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Actuals"), axis=1)

        test_mse = mean_squared_error(predictions_test["ActualDenormalize"], predictions_test["PredictionsDenormalize"])
        test_rmse = np.sqrt(test_mse)
        test_mape = mean_absolute_percentage_error(predictions_test["ActualDenormalize"], predictions_test["PredictionsDenormalize"]) *100
        test_r2 = r2_score(predictions_test["ActualDenormalize"], predictions_test["PredictionsDenormalize"])

        print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Train): {train_mse}, {train_rmse}, {train_r2}, {train_mape}")
        print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Val): {val_mse}, {val_rmse}, {val_r2}, {val_mape}")
        print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Test): {test_mse}, {test_rmse}, {test_r2}, {test_mape}")

        # Guardar el modelo y los hiperparámetros
        model.save(f'resultados/LSTMHyperparameter/{nombreArchivo[:-4]}_model_{i+1}.keras')

        # Guardar los hiperparámetros y las métricas en un archivo JSON
        hps_dict = hps.get_config()['values']
        optimizer_config = model.optimizer.get_config()
        hps_dict.update({
            'optimizer_config_type': optimizer_config["name"],
            'optimizer_config_learning_rate': float(optimizer_config["learning_rate"]),
            'batch_size': batch_size_LSTM,
            'mse_train': train_mse,
            'rmse_train': train_rmse,
            'r2_train': train_r2,
            'mape_train': train_mape,
            'mse_val': val_mse,
            'rmse_val': val_rmse,
            'r2_val': val_r2,
            'mape_val': val_mape,
            'mse_test': test_mse,
            'rmse_test': test_rmse,
            'r2_test': test_r2,
            'mape_test': test_mape
        })

    # Cargar el archivo JSON existente
    with open(f'resultados/LSTMHyperparameter/{nombreArchivo[:-4]}_model_{i+1}.json', 'w') as f:
        json.dump(hps_dict, f, indent=4)